In [1]:
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from nltk.metrics import BigramAssocMeasures
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [2]:
# first column is the review content (quoted)
# second column is the assigned sentiment (positive or negative)
def load_file():
    with open('D:\MS research\submissions\code\data/labelled_tweets_balanced.csv',encoding="utf8") as csv_file:
        reader = csv.reader(csv_file,delimiter=",",quotechar='"')
        next(reader)
        data =[]
        target = []
        for row in reader:
            # skip missing data
            if row[0] and row[1]:
                data.append(row[0])
                target.append(row[1])

        return data,target

In [ ]:
data, target = load_file()
print(data[0],target[0])

In [ ]:
# preprocess creates the term frequency matrix for the review data set
def preprocess_test(data):
    dat = data
    count_vectorizer = CountVectorizer(min_df=1,binary='false',ngram_range=(1,2))
    dat = count_vectorizer.fit_transform(dat)
    tfidf_data = TfidfTransformer(use_idf=False).transform(dat)

    return tfidf_data

In [ ]:
# preprocess creates the term frequency matrix for the review data set
def preprocess_tfidf(n):
    data,target = load_file()
    count_vectorizer = CountVectorizer(max_df=0.8,binary='true',ngram_range=(1,n))#,analyzer =text_process)
    data = count_vectorizer.fit_transform(data)
    tfidf_data = TfidfTransformer(use_idf=False).fit_transform(data)

    return tfidf_data

In [ ]:
# preprocess creates the term frequency matrix for the review data set
def preprocess_bow(n):
    dat,target = load_file()
    count_vectorizer = CountVectorizer(max_df=0.8,binary='false',ngram_range=(1,n))
    dat = count_vectorizer.fit_transform(dat)
    #tfidf_data = TfidfTransformer(use_idf=False).fit_transform(data)

    return dat

In [ ]:
def learn_model(data,target,classifier):
    data_train, target_train = data, target
    predicted = cross_val_predict(classifier,data_train, target_train, cv=10)
    #print(classification_report(target,predicted))
    evaluate_model(target_train,predicted)
    scores = cross_val_score(classifier,data_train,target_train,cv=10)
    print("10 Fold Cross Validation Scores for training data: ")
    print(scores)
    print()
    print("The mean score and the 95% confidence interval of the score estimate are given by:")
    print("Accuracy: %0.2f (+/- %0.2f)" %(scores.mean(), scores.std()*2))
    print()
    #target = label_binarize(target, classes=[0, 1, 2])
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    target_train = np.array([number[0] for number in lb.fit_transform(target_train)])
    recall = cross_val_score(classifier,data_train,target_train,cv=10, scoring='recall')
    print("10 Fold Cross Validation Recall: ")
    print(recall)
    print()
    #print("The mean score and the 95% confidence interval of the score estimate are given by:")
    print("Recall: %0.2f (+/- %0.2f)" %(recall.mean(), recall.std()*2))
    print()
    precision = cross_val_score(classifier,data_train,target_train,cv=10, scoring='precision')
    print("10 Fold Cross Validation Precision: ")
    print(precision)
    print()
    #print("The mean score and the 95% confidence interval of the score estimate are given by:")
    print("Precision: %0.2f (+/- %0.2f)" %(precision.mean(), precision.std()*2))
    print()
    
    #evaluate_model(target,predicted)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def evaluate_model(target_true,target_predicted):
    class_names=['1','2','3']
    print(classification_report(target_true,target_predicted))
    cnf_matrix = confusion_matrix(target_true, target_predicted)
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title='Confusion matrix')
    plt.show()
    #print('\nConfussion matrix:\n',confusion_matrix(target_true, target_predicted))
    print()
    print("The accuracy score is {:.2%}".format(accuracy_score(target_true,target_predicted)))
    print()

In [ ]:
def  Logistic():
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of logistic regression unigram tf idf:")
    LogisticRegression_classifier = LogisticRegression()
    print()
    learn_model(data_train,target_train,LogisticRegression_classifier)
    print()
    print("Testing results of logistic regression unigram tf idf")
    print()
    predicted = LogisticRegression_classifier.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of logistic regression unigram Term occurance:")
    LogisticRegression_classifier = LogisticRegression()
    print()
    learn_model(data_train,target_train,LogisticRegression_classifier)
    print()
    print("Testing results of logistic regression unigram Term occurance:")
    print()
    predicted = LogisticRegression_classifier.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    data,target = load_file()
    tfidf_data = preprocess_tfidf(2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of logistic regression bigram tf idf:")
    LogisticRegression_classifier = LogisticRegression()
    print()
    learn_model(data_train,target_train,LogisticRegression_classifier)
    print()
    print("Testing results of logistic regression bigram tf idf")
    print()
    predicted = LogisticRegression_classifier.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of logistic regression bigram Term occurance:")
    LogisticRegression_classifier = LogisticRegression()
    print()
    learn_model(data_train,target_train,LogisticRegression_classifier)
    print()
    print("Testing results of logistic regression bigram Term occurance:")
    print()
    predicted = LogisticRegression_classifier.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1+2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of logistic regression unigram+bigram tf idf:")
    LogisticRegression_classifier = LogisticRegression()
    print()
    learn_model(data_train,target_train,LogisticRegression_classifier)
    print()
    print("Testing results of logistic regression unigram+bigram tf idf")
    print()
    predicted = LogisticRegression_classifier.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1+2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of logistic regression unigram+bigram Term occurance:")
    LogisticRegression_classifier = LogisticRegression()
    print()
    learn_model(data_train,target_train,LogisticRegression_classifier)
    print()
    print("Testing results of logistic regression unigram+bigram Term occurance:")
    print()
    predicted = LogisticRegression_classifier.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
Logistic()

In [ ]:
def  KNN():
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of KNN k=1 unigram tf idf:")
    KNN_classifier1K = KNeighborsClassifier(n_neighbors=1)                       
    print()
    learn_model(data_train,target_train,KNN_classifier1K)
    print()
    print("Testing results of KNN k=1 unigram tf-idf:")
    print()
    predicted = KNN_classifier1K.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of KNN k=1 unigram Term occurance:")
    KNN_classifier1K = KNeighborsClassifier(n_neighbors=1)                       
    print()
    learn_model(data_train,target_train,KNN_classifier1K)
    print()
    print("Testing results of KNN k=1 unigram Term occurance:")
    print()
    predicted = KNN_classifier1K.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------") 
    data,target = load_file()
    tfidf_data = preprocess_tfidf(2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of KNN k=1 bigram tf idf:")
    KNN_classifier1K = KNeighborsClassifier(n_neighbors=1)                       
    print()
    learn_model(data_train,target_train,KNN_classifier1K)
    print()
    print("Testing results of KNN k=1 bigram tf-idf:")
    print()
    predicted = KNN_classifier1K.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of KNN k=1 bigram Term occurance:")
    KNN_classifier1K = KNeighborsClassifier(n_neighbors=1)                       
    print()
    learn_model(data_train,target_train,KNN_classifier1K)
    print()
    print("Testing results of KNN k=1 bigram Term occurance:")
    print()
    predicted = KNN_classifier1K.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------") 
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1+2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of KNN k=1 unigram+bigram tf idf:")
    KNN_classifier1K = KNeighborsClassifier(n_neighbors=1)                       
    print()
    learn_model(data_train,target_train,KNN_classifier1K)
    print()
    print("Testing results of KNN k=1 unigram+bigram tf-idf:")
    print()
    predicted = KNN_classifier1K.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1+2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of KNN k=1 unigram+bigram Term occurance:")
    KNN_classifier1K = KNeighborsClassifier(n_neighbors=1)                       
    print()
    learn_model(data_train,target_train,KNN_classifier1K)
    print()
    print("Testing results of KNN k=1 unigram+bigram Term occurance:")
    print()
    predicted = KNN_classifier1K.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------") 
KNN()

In [ ]:
def  Logistic():
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of unigram tf idf LogisticRegression with L2 penalty, Regularization Parameter C:100")
    classifier_l2_LR = LogisticRegression(C=100, penalty='l2', n_jobs=-1)
    print()
    learn_model(data_train,target_train,classifier_l2_LR)
    print()
    print("Testing results of unigram tf idf LogisticRegression with L2 penalty, Regularization Parameter C:100")
    print()
    predicted = classifier_l2_LR.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results unigram Term occurance LogisticRegression with L2 penalty, Regularization Parameter C:1")
    classifier_l2_LR = LogisticRegression(C=1, penalty='l2', n_jobs=-1)
    print()
    learn_model(data_train,target_train,classifier_l2_LR)
    print()
    print("Testing results unigram Term occurance LogisticRegression with L2 penalty, Regularization Parameter C:1")
    print()
    predicted = classifier_l2_LR.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    data,target = load_file()
    tfidf_data = preprocess_tfidf(2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of bigram tf idf with LogisticRegression with L2 penalty, Regularization Parameter C:100")
    classifier_l2_LR = LogisticRegression(C=100, penalty='l2', n_jobs=-1)
    print()
    learn_model(data_train,target_train,classifier_l2_LR)
    print()
    print("Testing results of bigram tf idf LogisticRegression with L2 penalty, Regularization Parameter C:100")
    print()
    predicted = classifier_l2_LR.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of bigram term occurance with LogisticRegression with L2 penalty, Regularization Parameter C:1")
    classifier_l2_LR = LogisticRegression(C=1, penalty='l2', n_jobs=-1)
    print()
    learn_model(data_train,target_train,classifier_l2_LR)
    print()
    print("Testing results of bigram term occurance with LogisticRegression with L2 penalty, Regularization Parameter C:1")
    print()
    predicted = classifier_l2_LR.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1+2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of unigram+bigram tf idf LogisticRegression with L2 penalty, Regularization Parameter C:100")
    classifier_l2_LR = LogisticRegression(C=100, penalty='l2', n_jobs=-1)
    print()
    learn_model(data_train,target_train,classifier_l2_LR)
    print()
    print("Testing results of unigram+bigram  tf idf LogisticRegression with L2 penalty, Regularization Parameter C:100")
    print()
    predicted = classifier_l2_LR.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1+2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results unigram+bigram  Term occurance LogisticRegression with L2 penalty, Regularization Parameter C:1")
    classifier_l2_LR = LogisticRegression(C=1, penalty='l2', n_jobs=-1)
    print()
    learn_model(data_train,target_train,classifier_l2_LR)
    print()
    print("Testing results unigram+bigram  Term occurance LogisticRegression with L2 penalty, Regularization Parameter C:1")
    print()
    predicted = classifier_l2_LR.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
Logistic()

In [ ]:
def SVC_Linear():
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of SVM linear unigram tf idf C=1:")
    SVC_classifier_linearKernel = SVC(C=1, kernel='linear')
    print()
    learn_model(data_train,target_train,SVC_classifier_linearKernel)
    print()
    print("Testing results of SVM linear unigram tf-idf  C=1:")
    print()
    predicted = SVC_classifier_linearKernel.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of SVM linear unigram Term occurance  C=0.1:")
    SVC_classifier_linearKernel = SVC(C=0.1, kernel='linear')
    print()
    learn_model(data_train,target_train,SVC_classifier_linearKernel)
    print()
    print("Testing results of SVM linear unigram Term occurance  C=0.1:")
    print()
    predicted = SVC_classifier_linearKernel.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    data,target = load_file()
    tfidf_data = preprocess_tfidf(2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of SVM linear bigram tf idf  C=1:")
    SVC_classifier_linearKernel = SVC(C=1, kernel='linear')
    print()
    learn_model(data_train,target_train,SVC_classifier_linearKernel)
    print()
    print("Testing results of SVM linear bigram tf-idf  C=1:")
    print()
    predicted = SVC_classifier_linearKernel.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of SVM linear bigram Term occurance C=0.1:")
    SVC_classifier_linearKernel = SVC(C=0.1, kernel='linear')
    print()
    learn_model(data_train,target_train,SVC_classifier_linearKernel)
    print()
    print("Testing results of SVM linear bigram Term occurance C=0.1:")
    print()
    predicted = SVC_classifier_linearKernel.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------") 
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1+2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of SVM linear unigram+bigram tf idf  C=10:")
    SVC_classifier_linearKernel = SVC(C=1, kernel='linear')
    print()
    learn_model(data_train,target_train,SVC_classifier_linearKernel)
    print()
    print("Testing results of SVM linear unigram+bigram tf-idf  C=1:")
    print()
    predicted = SVC_classifier_linearKernel.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1+2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of SVM linear unigram+bigram Term occurance C=0.1:")
    SVC_classifier_linearKernel = SVC(C=0.1, kernel='linear')
    print()
    learn_model(data_train,target_train,SVC_classifier_linearKernel)
    print()
    print("Testing results of SVM linear unigram+bigram Term occurance C=0.1:")
    print()
    predicted = SVC_classifier_linearKernel.fit(data_train,target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------") 
SVC_Linear()

In [ ]:
def SVC_RBF():
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of unigram tf idf SVM RBF kernel with C=10, gamma=1")
    SVC_classifier_rbfKernel = SVC(C=10,gamma=1, kernel='rbf')
    print()
    learn_model(data_train,target_train,SVC_classifier_rbfKernel)
    print()
    print("Testing results of unigram tf idf SVM RBF kernel with C=10, gamma=1")
    print()
    predicted = SVC_classifier_rbfKernel.fit(data_train, target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of unigram term occurance SVM RBF kernel with C=10, gamma=0.01")
    SVC_classifier_rbfKernel = SVC(C=10,gamma=0.01, kernel='rbf')
    print()
    learn_model(data_train,target_train,SVC_classifier_rbfKernel)
    print()
    print("Testing results of unigram  term occurance SVM RBF kernel with C=10, gamma=0.01")
    print()
    predicted = SVC_classifier_rbfKernel.fit(data_train, target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    data,target = load_file()
    tfidf_data = preprocess_tfidf(2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of bigram tf idf SVM RBF kernel with C=10, gamma=1")
    SVC_classifier_rbfKernel = SVC(C=10,gamma=1, kernel='rbf')
    print()
    learn_model(data_train,target_train,SVC_classifier_rbfKernel)
    print()
    print("Testing results of bigram tf idf SVM RBF kernel with C=10, gamma=1")
    print()
    predicted = SVC_classifier_rbfKernel.fit(data_train, target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of bigram term occurance SVM RBF kernel with C=10, gamma=0.01")
    SVC_classifier_rbfKernel = SVC(C=10,gamma=0.01, kernel='rbf')
    print()
    learn_model(data_train,target_train,SVC_classifier_rbfKernel)
    print()
    print("Testing results of bigram  term occurance SVM RBF kernel with C=10, gamma=0.01")
    print()
    predicted = SVC_classifier_rbfKernel.fit(data_train, target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    data,target = load_file()
    tfidf_data = preprocess_tfidf(1+2)
    data_train, data_test, target_train, target_test = train_test_split(tfidf_data, target, test_size=0.2, random_state=42)
    print("Training results of unigram+bigram tf idf SVM RBF kernel with C=10, gamma=1")
    SVC_classifier_rbfKernel = SVC(C=10,gamma=1, kernel='rbf')
    print()
    learn_model(data_train,target_train,SVC_classifier_rbfKernel)
    print()
    print("Testing results of unigram+bigram tf idf SVM RBF kernel with C=10, gamma=1")
    print()
    predicted = SVC_classifier_rbfKernel.fit(data_train, target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    bow=preprocess_bow(1+2)
    data_train, data_test, target_train, target_test = train_test_split(bow, target, test_size=0.2, random_state=42)
    print("Training results of bigram term occurance SVM RBF kernel with C=10, gamma=0.01")
    SVC_classifier_rbfKernel = SVC(C=10,gamma=0.01, kernel='rbf')
    print()
    learn_model(data_train,target_train,SVC_classifier_rbfKernel)
    print()
    print("Testing results of bigram  term occurance SVM RBF kernel with C=10, gamma=0.01")
    print()
    predicted = SVC_classifier_rbfKernel.fit(data_train, target_train).predict(data_test)
    #print(classification_report(target_test,predicted))
    evaluate_model(target_test,predicted)
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
SVC_RBF()